In [1]:
import pandas as pd
import datetime
from dateutil import tz
import os
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
conn = sqlite3.connect("databases/data")
%matplotlib inline

In [2]:
# this is the number of games we have to work with (incldues duplicates)
battle_logs = pd.read_sql("SELECT * FROM battle_logs", conn)
battle_logs = battle_logs[(battle_logs["mode"] == "brawlBall") & (battle_logs["map"] == "Sunny Soccer")]
len(battle_logs)

1342700

In [3]:
# this is the number of games we have to work with (no duplicates)

def clean_df(df):
    df["hash"] = df["time"].apply(hash) + df["mode"].apply(hash) + df["map"].apply(hash)
    df = df.drop_duplicates("hash")
    df["time"] = pd.to_datetime(df["time"], utc = True)
    df["time"] = df["time"].dt.tz_convert(tz.tzlocal())
    df = df[(df[["team1_player1_trophies", "team1_player2_trophies", "team1_player3_trophies"]].max(axis = 1) > 600)          & (df[["team2_player1_trophies", "team2_player2_trophies", "team2_player3_trophies"]].max(axis = 1) > 600)]
    return df
battle_logs = clean_df(battle_logs)
battle_logs.drop(["time", "mode", "map", "type", "duration", "hash"], axis = 1, inplace = True)
len(battle_logs)

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


33086

In [4]:
# making the result column (the column we will try to predict)
def first_team_result(row):
    """
    only filp the result if the source player is in team2
    """
    if row["source"] in [row["team2_player1_tag"], row["team2_player2_tag"], row["team2_player3_tag"]]:
        if row["result"] == "victory":
            return "defeat"
        if row["result"] == "defeat":
            return "victory"
        return "draw"
    else:
        return row["result"]
battle_logs["result"] = battle_logs.apply(first_team_result, axis = 1)

In [5]:
# access the player data
player_data = pd.read_sql("SELECT * FROM players",conn)
player_data.sort_values(["tag", "time_accessed"], ascending = False, inplace = True)
player_data.drop_duplicates("tag", inplace = True)

player_brawler_data = pd.read_sql("SELECT * FROM brawler_highest_trophies", conn)
player_brawler_data.sort_values(["player_id", "name", "time_accessed"], ascending = False, inplace = True)
player_brawler_data.drop_duplicates(["player_id", "name"], inplace = True)

In [6]:
# append the player data to the match df
def add_attribute_from_player_table(df, new_attributes, team_num, player_num):
    new_attributes["tag"] = "tag"
    player_data_attribute = player_data[list(new_attributes.keys())]
    for attribute in new_attributes:
        new_attribute = "team{}_player{}_{}".format(team_num, player_num, new_attributes[attribute])
        player_data_attribute.rename(columns = {attribute:new_attribute}, inplace = True)
    on = "team{}_player{}_tag".format(team_num, player_num)
    return df.merge(player_data_attribute, on = on, how = "left")

def add_highest_brawler_trophies(df, team_num, player_num):
    player_brawler_df = player_brawler_data[["player_id", "name", "highestTrophies"]]
    new_columns = {
        "player_id": "team{}_player{}_tag".format(team_num, player_num),
        "name": "team{}_player{}_brawler".format(team_num, player_num),
        "highestTrophies": "team{}_player{}_highestBrawlerTrophies".format(team_num, player_num)
    }
    player_brawler_df.rename(columns = new_columns, inplace = True)
    on = ["team{}_player{}_tag".format(team_num, player_num), "team{}_player{}_brawler".format(team_num, player_num)]
    return df.merge(player_brawler_df, on = on, how = "left")

new_attributes = {
    "highestTrophies": "highestTotalTrophies",
    "trophies": "totalTrophies",
    "expPoints": "exp",
    "highestPowerPlayPoints": "highestPowerPlay",
    "3vs3Victories": "3vs3Victories",
    "soloVictories": "soloVictories",
    "duoVictories": "duoVictories"
}

for team_num in [1,2]:
    for player_num in [1,2,3]:
        battle_logs = add_attribute_from_player_table(battle_logs, new_attributes, team_num, player_num)
        battle_logs = add_highest_brawler_trophies(battle_logs, team_num, player_num)

C:\Users\student\Anaconda3\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [7]:
battle_logs.to_csv("data/full_battle_log.csv")